In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Data
data = {
    "text": [
        "I love this product, it’s fantastic!",
        "Worst service I have ever experienced.",
        "The movie was really enjoyable and fun.",
        "I hated the food, it was terrible.",
        "Great experience, would recommend!",
        "Awful, I will never come back.",
         "Great experience, would recommend!",
        "Awful, I will never come back.",
         "Great experience, would recommend!",
        "Awful, I will never come back.",
         "Great experience, would recommend!",
        "Awful, I will never come back.",
    ],
    "label": ["positive", "negative", "positive", "negative", "positive", "negative",
              "positive", "negative",
              "positive", "negative",
              "positive", "negative"]
}
df = pd.DataFrame(data)
encoder = LabelEncoder()
df["label_enc"] = encoder.fit_transform(df["label"])

X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label_enc"], test_size=0.3, random_state=42
)

# Load USE as a callable model
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Precompute embeddings
X_train_emb = use_model(X_train).numpy()
X_test_emb = use_model(X_test).numpy()

# Build a simple classifier
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(512,)),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train_emb, y_train, epochs=10, batch_size=3, validation_data=(X_test_emb, y_test))

preds = model.predict(X_test_emb)
print("Predicted probs:", preds.flatten())
